## On essaye de faire un truc qui marche ajd

In [10]:
from Bio import Entrez, SeqIO


with open("GeneSymbols_45.txt", "r") as infos:
    for line in infos:
        symbol = line.split(",")[0]
        current_species = line[:-1].split(",")[1]
        species_info= {"species" : current_species, "gene_symbol" : symbol}

        
        species_info = Info(species_info)
        print(species_info)  

 

{'species': 'homo_sapiens', 'gene_symbol': 'RAD51', 'official_name': 'RAD51 recombinase', 'gene_id': '5888', 'transcript_id': ['NM_001164270.2', 'NM_001164269.2', 'NM_133487.4', 'NM_002875.5', 'XM_054378571.1', 'XM_054378570.1', 'XM_054378569.1', 'XM_054378568.1', 'XM_054378567.1', 'XM_054378566.1', 'XM_047432925.1', 'XM_011521861.3', 'XM_011521857.3', 'XM_011521860.3', 'XM_011521859.3', 'XM_011521858.3'], 'prot_id': ['NP_001157742.1', 'NP_001157741.1', 'NP_597994.3', 'NP_002866.2', 'XP_054234546.1', 'XP_054234545.1', 'XP_054234544.1', 'XP_054234543.1', 'XP_054234542.1', 'XP_054234541.1', 'XP_047288881.1', 'XP_011520163.2', 'XP_011520159.2', 'XP_011520162.1', 'XP_011520161.1', 'XP_011520160.1']}
{'species': 'arabidopsis_thaliana', 'gene_symbol': 'PHYB', 'official_name': 'phytochrome B', 'gene_id': '816394', 'transcript_id': ['NM_001335612.1', 'NM_127435.4'], 'prot_id': ['NP_001325249.1', 'NP_179469.1']}
{'species': 'saccharomyces_cerevisiae', 'gene_symbol': 'SFA1', 'official_name': 'bi

## Aggrégateur

In [8]:
def Info (species_info):
    species_info = Gene(species_info)
    species_info = ProTranscript(species_info)
    return(species_info)
    

## Requête

In [5]:
from Bio import Entrez, SeqIO

def Request (species_info, db) :

    Entrez.email = "Chevreau.julien21@gmail.com@univ-rouen.fr"

    # Input pour la requête
    term = "%s[Orgn] AND %s[gene]" % (species_info["species"], species_info["gene_symbol"])

    # Va chercher les IDs dans la base de donnée demandée
    handle = Entrez.esearch(db=db, term=term, retmax=100)
    records = Entrez.read(handle)

    identifiers = records["IdList"]

    # Cherche les données à partir des IDs. Résultat en format GenBank
    handle = Entrez.efetch(db=db, id=identifiers, retmax="100",
                            rettype="gbk", retmode="text")
    text = handle.read()

    # Construit un fichier temporaire pour être lu ensuite
    filename = "tmp_%s.gbk" % (db)
    with open(filename,"w") as gbk:
        gbk.write(text)

    return(filename)


## Prot et transcrits traduits 
Aller chercher dans la base **prot** pour avoir les différents IDs des prots et transcrits associés

In [ ]:
species_info = {'species': 'homo_sapiens', 'gene_symbol': 'RAD51'}

def ProTranscript (species_info): 
    prot_file = Request(species_info, "protein")

    for seq_record in SeqIO.parse(prot_file, "genbank"):
        db_source = seq_record.annotations["db_source"].split(" ")

        if db_source[0] == "REFSEQ:" :
            if "transcript_id" not in species_info :
                species_info["transcript_id"] = [db_source[-1]]
                species_info["prot_id"] = [seq_record.id]
            else :
                species_info["transcript_id"].append(db_source[-1])
                species_info["prot_id"].append(seq_record.id)
    return(species_info)

ProTranscript(species_info)

## Transcrits non traduits
Aller dans **nucleotide** pour les transcrits non traduits

In [ ]:
species_info = {'species': 'homo_sapiens', 'gene_symbol': 'RAD51', 'transcript_id': ['NM_001164270.2', 'NM_001164269.2', 'NM_133487.4', 'NM_002875.5', 'XM_054378571.1', 'XM_054378570.1', 'XM_054378569.1', 'XM_054378568.1', 'XM_054378567.1', 'XM_054378566.1', 'XM_047432925.1', 'XM_011521861.3', 'XM_011521857.3', 'XM_011521860.3', 'XM_011521859.3', 'XM_011521858.3'], 'prot_id': ['NP_001157742.1', 'NP_001157741.1', 'NP_597994.3', 'NP_002866.2', 'XP_054234546.1', 'XP_054234545.1', 'XP_054234544.1', 'XP_054234543.1', 'XP_054234542.1', 'XP_054234541.1', 'XP_047288881.1', 'XP_011520163.2', 'XP_011520159.2', 'XP_011520162.1', 'XP_011520161.1', 'XP_011520160.1']}

def NCrna(species_info) :

    transcript_file = Request(species_info,"nucleotide")

    for seq_record in SeqIO.parse(transcript_file, "genbank"):
        print(seq_record.annotations["molecule_type"])
        if seq_record.annotations["molecule_type"] == "mrna" :
            if not seq_record.id in species_info['transcript_id']:
                species_info['transcript_id'].append(seq_record.id + " (Not translated)")
    
    return(species_info)

NCrna(species_info)

## Gène
Aller dans **gene** pour l'official full name et le gene access number

In [57]:
species_info = {'species': 'escherichia_coli', 'gene_symbol': 'HYBE'}
def Gene (species_info) :
    import re

    gene_file = Request(species_info,"gene")

    with open(gene_file,"r") as file:
        content = "".join(file.readlines())
        entry_one = re.search(r"\n(.*)\n\n2\.", content, flags=re.S).group(1)
        print(entry_one)

        pattern_id = re.search(r"ID:\s(\d*)",entry_one)
        pattern_official = re.search(r"Name:\s(.*)\s\[",entry_one)
        pattern_official_alt = re.search(r"(.*)\s\[.*\]",entry_one)

        if pattern_official != None :
            species_info["official_name"] = pattern_official.group(1)
        elif pattern_official_alt != None :
            species_info["official_name"] = pattern_official_alt.group(1)

        if pattern_id != None :
            species_info["gene_id"] = pattern_id.group(1)
    return(species_info)

Gene(species_info)


1. hybE
Official Symbol: hybE and Name: hydrogenase 2-specific chaperone [Escherichia coli str. K-12 substr. MG1655]
Other Aliases: b2992, ECK2986
Other Designations: hydrogenase 2-specific chaperone
Annotation:  NC_000913.3 (3140311..3140799, complement)
ID: 947483


{'species': 'escherichia_coli',
 'gene_symbol': 'HYBE',
 'official_name': 'hydrogenase 2-specific chaperone',
 'gene_id': '947483'}

## Section Test

In [49]:
import re
with open ("tmp_gene.gbk", "r") as file :
    contenu = "".join(file.readlines())
    entry_one = re.search(r"\n(.*)\n\n2\.", contenu, flags=re.S)
    
    print (contenu)

1. hybE
Official Symbol: hybE and Name: hydrogenase 2-specific chaperone [Escherichia coli str. K-12 substr. MG1655]
Other Aliases: b2992, ECK2986
Other Designations: hydrogenase 2-specific chaperone
Annotation:  NC_000913.3 (3140311..3140799, complement)
ID: 947483

1. hybE
Official Symbol: hybE and Name: hydrogenase 2-specific chaperone [Escherichia coli str. K-12 substr. MG1655]
Other Aliases: b2992, ECK2986
Other Designations: hydrogenase 2-specific chaperone
Annotation:  NC_000913.3 (3140311..3140799, complement)
ID: 947483

2. hybE
hydrogenase 2-specific chaperone [Escherichia coli O157:H7 str. Sakai]
Other Aliases: ECs_3877
Other Designations: hydrogenase 2-specific chaperone
Annotation:  NC_002695.2 (3880792..3881280, complement)
ID: 916298

3. hybE
hydrogenase-2 assembly chaperone [Escherichia coli DSM 30083 = JCM 1649 = ATCC 11775]
Other Aliases: EAS44_RS04475, EAS44_20040
Other Designations: hydrogenase-2 assembly chaperone
This record was discontinued.
ID: 49583044

4. hybE

## Conversion gbk en fasta

In [ ]:
from Bio import SeqIO
SeqIO.convert("all_from_eu.gbk",'genbank',"all_from_eu.fasta","fasta")

In [ ]:
from Bio import SeqIO
gb_file = "f1.gbk"
features = {}
genes = {}
for seq_record in SeqIO.parse(gb_file, "genbank"):
    for feat in seq_record.features :
        if feat.type in features :
            features[feat.type]+=1
        else : 
            features[feat.type] = 1
        if "gene" in feat.qualifiers:
            for gene in feat.qualifiers["gene"]:
                genes[gene] = 1
                
print(features)
print(genes)